In [7]:
import sys
import os
import json
import pickle

from datetime import datetime

from sklearn.svm import SVC
from scipy import interpolate as inp
import numpy as np

sys.path.append('../')
from predict_square.utils import interpolate, normalize, skinny
from predict_square.constant import ROOT_DIR, DATA_DIR, MODEL_DIR

## Load Data

In [8]:
with open(os.path.join(DATA_DIR, 'square.json'),'r') as f:
    square_data = json.load(f)
    
with open(os.path.join(DATA_DIR, 'notsquare.json'),'r') as f:
    notsquare_data = json.load(f)
    

## Preprocess Data

In [9]:
X_square, X_not_square = [], []
for i in range(len(square_data["data"])):
    normalized_data = normalize(square_data["data"][i])
    interpolated_data = interpolate(normalized_data, 100)
    skinny_data = skinny(interpolated_data)
    X_square.append(skinny_data)
    
for i in range(len(notsquare_data["data"])):
    normalized_data = normalize(notsquare_data["data"][i])
    interpolated_data = interpolate(normalized_data, 100)
    skinny_data = skinny(interpolated_data)
    X_not_square.append(skinny_data)

In [10]:
X = X_square + X_not_square

In [11]:
y = ['square'] * 100 + ['nsquare'] * 100

## Train Model

In [12]:
clf = SVC(kernel='linear', random_state=42, probability=True)
clf.fit(X, y)

model_name = 'svm'
version = datetime.now().strftime('%Y%m%d%H%M')

svm_filename = os.path.join(MODEL_DIR, model_name, version)
pickle.dump(clf, open(svm_filename, 'wb'))

## Predict using seen data

In [13]:
from predict_square.predict import Predictor

p = Predictor()
for i in range(40):
    print("Actual: Not Square\nPredicted: {}\n".format(p.predict_proba(notsquare_data["data"][i])))
    print("Actual: Square\nPredicted: {}\n".format(p.predict_proba(square_data["data"][i])))

FileNotFoundError: [Errno 2] No such file or directory: '/opt/predict_square/models/svm'